In [26]:
import numpy as np
from math import sqrt
import pandas as pd

In [27]:
class UnlimitedDataWorks:

    def __init__(self, deg):
        self.exp = []
        for i in range(deg+1):
            for j in range(deg+1):
                if i+j <= deg:
                    self.exp.append((i, j))

    def train_test_split(self, dataframe):
        normalize = lambda x: ((x - x.min()) / (x.max() - x.min()))
        self.data = pd.DataFrame([])
        self.count = -1
        for (a, b) in self.exp:
            self.count += 1
            res = (dataframe["lat"] ** a) * (dataframe["lon"] ** b)
            self.data.insert(self.count, "col" + str(a) + str(b), res, True)

        self.count += 1 
        dataframe = normalize(dataframe)
        self.data = normalize(self.data)
        self.data["col00"] = [1.0]*len(self.data)

        # generate a 70-20-10 split on the data:
        X = self.data[:304113]
        Y = dataframe["alt"][:304113]
        xval = self.data[304113:391088]
        yval = dataframe["alt"][304113:391088]
        x = self.data[391088:]
        y = dataframe["alt"][391088:]   
        return (X, Y, xval, yval, x, y)

In [28]:
class RegressionModel:

    def __init__(self, N, X, Y, x, y, xval, yval):
        """
        X :: training data                  (304113 x 3)
        x :: testing data                   (43786 x 3)
        Y :: training target values         (304113 x 1)
        y :: testing target values          (43786 x 1)
        xval :: validation data             (86975 x 3)
        yval :: validation training data    (86975 X 1)
        """
        self.N = N
        self.X = np.array(X)
        self.Y = np.array(Y)
        self.x = np.array(x)
        self.y = np.array(y)
        self.xval = np.array(xval)
        self.yval = np.array(yval)

    def score(self, weights):
        """
        the following method helps us find the
        R2 (R-squared) error of a given training data
        wrt the generated weights
        """
        ss_tot = sum(np.square(np.mean(self.y) - self.y))
        ss_res = sum(np.square((self.x @ weights) - self.y))
        rmse = sqrt(ss_res/len(self.x))
        r2 = (1-(ss_res / ss_tot))
        return [r2*100, rmse]

    def gradient_descent(self):
        """
        train till error is almost constant
        """
        lr = 8e-7
        prev_err, count = 1e10, 0
        W = np.random.randn(self.N)
        while True:
            diff = ((self.X @ W) - self.Y)
            err = 0.5 * (diff @ diff)
            grad = (self.X.T @ diff)
            if count % 500 == 0:
                print("epoch =", count, "| err_diff =", prev_err-err)
                print("error = ", err, "||", W)
                print("score =", self.score(W), end="\n\n")
            W -= lr * grad
            if abs(prev_err-err) <= 1e-4:
                break
            prev_err = err
            count += 1
        print(count, err)
        print(W, self.score(W), end="\n\n")

    def gradient_descent_L1_reg(self):
        """
        attempts a L1 regularization on the data
        considering 10% of training data as validation data
        """
        W_fin = np.array([])
        lr, l1_fin = 8e-7, 0
        MVLE = 1e10
        L1_vals = [0.0, 0.05, 0.15, 0.25, 0.35, 0.45,
                   0.55, 0.65, 0.75, 0.85, 0.95, 1.0]
        sgn = lambda x: (x / abs(x))
        for l1 in L1_vals:
            prev_err, count = 1e10, 0
            W = np.random.randn(self.N)
            while True:
                diff = ((self.X @ W) - self.Y)
                err = 0.5 * ((diff @ diff) + l1*sum([abs(w) for w in W]))
                if count % 500 == 0:
                    print("L1 hyperparamter =", l1, end=", ")
                    print("epoch =", count, "| err_diff =", prev_err-err)
                    print("error = ", err, "||", W)
                    print("score =", self.score(W), end="\n\n")
                sgn_w = np.array([sgn(w) for w in W])
                W -= lr * ((self.X.T @ diff) + 0.5*l1*sgn_w)
                if abs(prev_err-err) <= 0.01:
                    break
                prev_err = err
                count += 1
            VLD = ((self.xval @ W) - self.yval)
            VLE = 0.5 * ((VLD.T @ VLD) + l1*sum([abs(w) for w in W]))
            if VLE < MVLE:
                W_fin = W
                l1_fin = l1
                MVLE = VLE
        print(MVLE, l1_fin, W_fin)

    def gradient_descent_L2_reg(self):
        """
        attempts a L2 regularization on the data
        considering 10% of training data as validation data
        """
        W_fin = np.array([])
        lr, l2_fin = 5e-7, 0
        MVLE = 1e10
        L2_vals = [0, 0.05, 0.15, 0.25, 0.35, 0.45,
                   0.55, 0.65, 0.75, 0.85, 0.95, 1.0]
        for l2 in L2_vals:
            prev_err, count = 1e10, 0
            W = np.random.randn(self.N)
            while True:
                diff = ((self.X @ W) - self.Y)
                err = 0.5 * ((diff @ diff) + l2*sum([w*w for w in W]))
                if count % 500 == 0:
                    print("L2 hyperparamter =", l2, end=", ")
                    print("epoch =", count, "| err_diff =", prev_err-err)
                    print("error = ", err, "||", W)
                    print("score =", self.score(W), end="\n\n")
                W -= lr * ((self.X.T @ diff) + l2*W)
                if abs(prev_err-err) <= 0.01:
                    break
                prev_err = err
                count += 1
            VLD = ((self.xval @ W) - self.yval)
            VLE = 0.5 * ((VLD.T @ VLD) + l2 * (W.T @ W))
            if VLE < MVLE:
                W_fin = W
                l2_fin = l2
                MVLE = VLE
        print(MVLE, l2_fin, W_fin)

    def fit(self):
        """
        solves for optimal weights using system of
        N linear equations; AW = B, hence, W = inv(A)*B
        """
        B = self.X.T @ self.Y
        A = self.X.T @ self.X
        W = (np.linalg.inv(A)) @ B
        print(W, self.score(W))
        # diff = ((self.X @ W) - self.Y)
        # err = 0.5 * (diff @ diff)
        # print("train_error =", err)

In [29]:
columns = ["junk", "lat", "lon", "alt"]
raw_df = pd.read_csv("3D_spatial_network.txt", sep=',', header=None,
                     names=columns).drop("junk", 1)

deg = input("Enter the Degree of the Polynomial:")
pre_processor = UnlimitedDataWorks(deg=int(deg))
X_train, Y_train, x_val, y_val, x_test, y_test = pre_processor.train_test_split(raw_df)

model = RegressionModel(N=pre_processor.count,
                        X=X_train,
                        Y=Y_train,
                        x=x_test,
                        y=y_test,
                        xval=x_val,
                        yval=y_val)

model.fit()
# model.gradient_descent()
# model.gradient_descent_L1_reg()
model.gradient_descent_L2_reg()

Enter the Degree of the Polynomial:6
[-1.37916138e+02  2.98353613e+09 -1.27841774e+10  2.13668156e+10
 -1.72308151e+10  6.58022130e+09 -9.15567932e+08 -2.41751000e+05
  9.08304562e+05 -1.60691625e+06  2.12112309e+06 -1.68191375e+06
  3.20636266e+05  3.59078000e+05 -9.48474750e+05  2.48719469e+05
  6.47351188e+05  4.91456750e+05 -6.68262500e+04  8.24956625e+05
 -1.70735775e+06 -5.99249375e+05 -4.65710250e+05  1.58675750e+06
  4.25131953e+05 -2.99192000e+05 -4.84041125e+05  1.60199812e+05] [-5373.962513447813, 0.9693734944216973]
L2 hyperparamter = 0, epoch = 0 | err_diff = 9999391672.058455
error =  608327.9415461808 || [-0.43931265 -0.70112211 -0.25107395  0.33731852 -1.51885897  0.17611446
  0.50646478  1.39241768  1.03569587  1.0320072  -0.74321001 -0.01523486
  0.17978389  1.07976341 -0.37893446 -0.89429821  0.95327667 -0.8407187
 -1.29064996 -1.47351431  0.45135585 -0.4754581  -0.16230245 -0.63648778
 -0.77062959 -0.45978376 -0.21951029  0.34446643]
score = [-23102.782760239, 1.995

L2 hyperparamter = 0, epoch = 4000 | err_diff = 0.011655582200091885
error =  2453.4542409003298 || [ 0.07816513 -0.4718611  -0.01914162  0.57189283 -1.28167231  0.41588357
  0.74878612  1.29138731  0.96041877  0.98109031 -0.77107389 -0.02122019
  0.19461828  1.08244367 -0.3608943  -0.86123551  1.00104424 -0.77856402
 -1.18973219 -1.36073442  0.57585013 -0.33938461  0.02989004 -0.43422607
 -0.55838079 -0.18443765  0.06475792  0.69404621]
score = [4.606069352892672, 0.1279677703715684]

L2 hyperparamter = 0.05, epoch = 0 | err_diff = 9999120822.11964
error =  879177.8803598003 || [ 0.09445455 -1.36432789  0.13540772  0.10717697 -0.68985941 -1.26259617
  0.52648857  0.18151194 -0.73889356  0.82122612  1.52202835  1.05864463
 -1.24646153 -0.25735354  1.62146549 -0.69036755  0.29449425 -0.70721221
  0.15675847 -2.3621672  -1.66120739 -1.46656004  0.2207515   0.97907811
 -0.00446666 -0.59545464 -0.04857234  0.60944983]
score = [-33377.20319086073, 2.397258213518564]

L2 hyperparamter = 0.15

L2 hyperparamter = 0.35, epoch = 1500 | err_diff = 0.011800405302892614
error =  2433.1973701229354 || [ 0.19679795  0.63632817  0.62039861 -0.38154858 -1.20103491  0.36514616
 -0.8063756   0.91199476 -1.87713914 -0.03392283  1.10532722 -0.18800069
  1.87871582 -0.90450439 -3.03649622  0.40906556 -1.22870021  1.71069286
  1.77937322 -1.20464546  0.69436489  1.00159976  0.47209637  1.51369538
 -0.0250405  -2.41577097 -0.17222996  0.02388401]
score = [5.751187564036986, 0.12719738253031235]

L2 hyperparamter = 0.45, epoch = 0 | err_diff = 9999406903.52765
error =  593096.472350055 || [-0.59723686  1.89979951 -1.59219003 -1.14131773  0.45872025 -1.6447479
 -0.66601558  0.7420054  -0.20217875 -0.20281311 -0.78339468 -0.2570079
 -0.14188738 -1.39945757  0.56418374  0.58082923 -2.79699551 -1.10530458
 -0.51126755  0.8443011   1.24954449 -0.20766654  0.03579568  0.27703148
  1.48433745  1.55619217 -0.07268296  1.6803027 ]
score = [-22507.56019211839, 1.9700038688243038]

L2 hyperparamter = 0.

L2 hyperparamter = 0.45, epoch = 4000 | err_diff = 0.10135110958935911
error =  2592.367928958994 || [ 0.20469212  2.34436311 -1.14315136 -0.69138058  0.90849222 -1.19183953
 -0.21277387  1.24237652  0.28942209  0.28020651 -0.30754091  0.21148745
  0.3203433  -1.22843468  0.72560516  0.73484933 -2.64689649 -0.96330814
 -0.67334122  0.67368317  1.07150219 -0.39118358 -0.45501907 -0.22102967
  0.97834713  0.74452325 -0.88962564  0.56195838]
score = [-0.287670984006283, 0.13120911432804075]

L2 hyperparamter = 0.45, epoch = 4250 | err_diff = 0.08501056817613062
error =  2569.1408494649095 || [ 0.19657623  2.34203841 -1.14500827 -0.69299329  0.9070571  -1.19289098
 -0.21361694  1.25675325  0.30290488  0.29284039 -0.29563364  0.22268637
  0.3309294  -1.22181645  0.73163172  0.74042689 -2.6415459  -0.95843961
 -0.67494997  0.67167182  1.06915984 -0.39373496 -0.46499698 -0.23127029
  0.9678013   0.72609659 -0.90816244  0.53526604]
score = [0.5822052544031342, 0.13063883352105057]

L2 hyperpar

L2 hyperparamter = 0.55, epoch = 750 | err_diff = 0.013792656291570893
error =  2477.9244494770037 || [ 0.06742734  1.53864433 -1.67591934  1.58937562  1.860392   -1.71870001
 -1.54612127  0.65560767  0.32316176  0.53501885  0.27276462 -0.21971609
 -0.7517602   0.87639933  0.11175806  0.77383026 -0.0841066   0.53426408
  0.09771392 -2.61717551  0.68043937 -0.56684287  1.27018216  0.05899216
 -1.77666981 -1.48909546  0.93411716  0.32233765]
score = [4.05246138351637, 0.12833855649193696]



KeyboardInterrupt: 